In [2]:
# Imports
import numpy as np
import pandas as pd

# Image processing tools
import skimage.feature
import skimage.filters
import skimage.filters.rank
from skimage.filters import threshold_otsu
import skimage.io
import skimage.morphology
import skimage.segmentation
from scipy import ndimage as ndi
import scipy.signal

import glob
import os

import tqdm as tqdm

In [3]:
def brightness_counter(im_labeled, im_pos, im_height, im_width, template_height, template_width):
    '''
    Brightness counter is a function that produces a list of brightness values for up to one thousand cells in an image. 
    The brightness values is a sum of the intensity for a given labelled object.
    
    Parameters
    im_labeled: An image array with each independant object labeled with a separate value generated by skimage.measure.label
    im_pos: The result of the autofluorescence subtraction
    
    Returns
    cell_list: list of cells selected for analysis
    cell_intensities: list of cell intensities
    '''
    
    # Determine whether down-sampling is required 
    if np.max(im_labeled) > 1000:
        # Create a list of the cells to quantify the brightness in using a random number generator without replacement  
        cell_list = np.random.choice(np.arange(1, np.max(im_labeled)), 1000, replace=False)
    else:
        # If there are less than 100 cells, create a list of all the cells to access
        cell_list = np.arange(1, np.max(im_labeled) + 1)
    
    cell_intensities = {}
    
    # Create the cropped image to ensure that the image arrays are the same size. 
    im_pos_crop = im_pos[int(template_height/2 - 1):int(im_height - int(template_height/2)), int(template_width/2 - 1):int(im_width - int(template_width/2))]
    
    # Simplify labels to a Nx2 array, with an array of i and an array of j indices
    nonzero_inds = im_labeled.nonzero()
    
    # Fill in an array of the cell labels for later boolean logic
    labels = [im_labeled[i,j] for i,j in zip(nonzero_inds[0], nonzero_inds[1])]
    
    # Instantiate dictionary
    cell_intensities = {}
 
    # Loop through the prepared cell list
    for count_value in cell_list:
        
        # Get a list of i and of j indices
        label_bool = (labels == count_value)
        inds = (nonzero_inds[0][label_bool], nonzero_inds[1][label_bool])
        
        # In the positive image, sum over the whole cell to get its intensity
        intensities = im_pos_crop[inds]
        cell_intensity = np.sum(intensities)/len(intensities)
        
        # Append this to a list of all cell intensities
        cell_intensities[str(count_value)] = cell_intensity

    return(cell_list, cell_intensities)

In [4]:
def cell_counter(filepath, gaussian_size = 5, truncation = 2, threshold = 1000, size_thresh = 0.5, min_size = 10, interpixel_distance = 0.75488):
    # Get the list of files in the indicated folder 
    file_lst = glob.glob(filepath + "*.tif")
    
    # Get the list of files in the indicated folder 
    reduced_file_lst = glob.glob(filepath + "*CH1.tif")
   
    # Using the approved microscope channels, determine which channels are present
    channels = iris.get_channel_lists(file_lst) 
    
    # Filter to obtain the individual image channel files, excluding overlay 
    verified_files = [item for sublist in channels.values() for item in sublist]
    
    # Get the positions of the corners and the size of the files
    corners, size = iris.get_stitching(verified_files)
    
    # Initialize the unstitched image size 
    im_size = [1920, 1440]

    # Initialize the image channels
    im_channels = {'CH1' : 0,
                   'CH2' : 0}
    
    # Create blended images of each channel
    for channel in channels:
        im_channels[channel] = iris.blend(channels[channel], corners, size)

    # Determine the stitched image shape
    im_height, im_width = np.shape(im_channels['CH1'])

    # Obtain the red (autofluorescent) and green (signal) channels of an image
    im_sig = skimage.img_as_float(im_channels['CH1'])
    im_af = skimage.img_as_float(im_channels['CH2'])
    
    # Determine the threshold to discriminate tissue area from background
    area_thresh = threshold_otsu(im_af, nbins=1024)

    # We can threshold this image based on this intensity to get all pixels that comprise tissue
    threshed_im = im_af > area_thresh

    # By using this threshold, we can determine the total area
    total_area = sum(sum(threshed_im))*(interpixel_distance**2)
    
    # Unused variables are deleted to avoid memory error
    del im_channels, threshed_im
    
    # Determine the ratio between the mean signal intensity and the mean autofluorescence intensity
    multiplication_ratio = np.mean(im_sig)/np.mean(im_af)
    
    # Adjust the autofluorescent image by this mean signal and subtract the adjusted autofluorescent image from the signal image 
    im_sub = im_sig - im_af*multiplication_ratio

    del im_sig, im_af
    
    # Apply a gaussian blur to determine the background patterning and subtract this from the current image
    im_bg = skimage.filters.gaussian(im_sub, sigma = gaussian_size, truncate = truncation)

    im_no_bg = im_sub - im_bg

    # Determine the maximum and minimum values (returned to simplify troubleshoot pipeline on smaller images)
    min_val = im_no_bg.min()
    max_val = im_no_bg.max()
    
    # Normalize the image intensity values between one and zero
    im_no_bg = (im_no_bg - im_no_bg.min()) / (im_no_bg.max() - im_no_bg.min())

    del im_bg
    
    # Compute the histogram of the image intensities
    hist_bin = skimage.exposure.histogram(im_no_bg)
    hist, bins = hist_bin

    # Determine the second derivative of the gradient
    dn_dk = np.diff(np.diff(hist))
    
    # Using the thresh_finder function, determine the threshold that enables separation of background from bright objects
    thresh = thresh_finder(dn_dk, bins, desired_thresh = threshold)

    # We can then initialize a output image the same dimensions as im_sub
    output_file = np.zeros_like(im_sub)
    
    # We can then loop over all the verified files and perform size and intensity thresholding. Doing this on the smaller files allows the analysis of larger images with less memory  
    for f in tqdm.tqdm(reduced_file_lst):
        # Get the corner of the image
        corner = corners[f]
        
        # Determine the regions of the image that are brighter than the determined threshold. Perform this over the same range as the template, since the template cannot perform partial matching on the edges. 
        binary_gauss = im_no_bg[corner[1] + int(template_height/2 - 1):corner[1] + im_size[1] - int(template_height/2), corner[0] + int(template_width/2 - 1):corner[0] + im_size[0] - int(template_width/2)] > thresh 
        binary_gauss = skimage.morphology.binary_closing(binary_gauss, skimage.morphology.disk(3))
        
        # Perform feature matching to the template and threshold based on similarity
        im_feat = skimage.feature.match_template(im_sub[corner[1]:corner[1] + im_size[1], corner[0]:corner[0] + im_size[0]], template)
        binary_feat = im_feat > size_thresh

        # Multiply both thresholded masks together to get the objects that are both bright and circular
        binary_mult = binary_gauss*binary_feat

        # Remove small objects, usually pixels not associated with nuclear fluorescence
        binary_rem = skimage.morphology.remove_small_objects(binary_mult, min_size=min_size)

        # Write this information into the output file
        output_file[corner[1] + int(template_height/2 - 1):corner[1] + im_size[1] - int(template_height/2), corner[0] + int(template_width/2 - 1):corner[0] + im_size[0] - int(template_width/2)] = binary_rem

    del im_no_bg, binary_gauss, im_feat, binary_feat, binary_mult, binary_rem
    
    # trim the output file to remove the edges 
    trimmed_output_file = output_file[int(template_height/2 - 1):im_height - int(template_height/2), int(template_width/2 - 1):im_width - int(template_width/2)]
    
    # Count the number of labeled cells
    im_labeled, n_labels = skimage.measure.label(trimmed_output_file, background=0, return_num=True)
    
    # Determine the brightness of the cells (sampled to 1000 if more cells are counted)
    cell_list, cell_intensity_list = brightness_counter(im_labeled, im_sub, im_height, im_width, template_height, template_width)
    
    del im_sub, output_file, trimmed_output_file, im_labeled
    
    # return the number of cells, the list of cells brightness is determined on, the list of the corresponding brightness, the threshold applied, the multiplication ratio, and the normalization values
    return(n_labels, cell_list, cell_intensity_list, thresh, multiplication_ratio, min_val, max_val, area_thresh, total_area)
